In [1]:
## Preloads
using Statistics
using FFTW
using Distributed
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
using Plots
push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
using DHC_2DUtils
using MLDatasets
using Images
using HDF5
theme(:dark)

┌ Info: Precompiling DHC_2DUtils [top-level]
└ @ Base loading.jl:1278
┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1278


In [4]:
addprocs(7)

7-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8

In [5]:
@everywhere begin
    using Statistics
    using LinearAlgebra
    using Distributed
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using Images
    using Interpolations
    using DSP
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    # generate an image of a rod with some position, length, position angle,
    # and FWHM
    function rod_half(xcen, ycen, length, pa, fwhm)
        nx = 256
        xbox = 1:nx
        ybox = 1:nx
        xbox = xbox.-nx/2 .+xcen
        ybox = ybox.-nx/2 .+ycen

        sx  = xbox' .* ones(size(ybox)[1])
        sy  = ones(size(xbox)[1])' .* ybox

        sig = fwhm/2.355
        dtor = pi/180
        # -------- define a unit vector in direction of rod at position angle pa
        ux = cos(pa*dtor)   # 90 deg is up
        uy = sin(pa*dtor)   # 0 deg to the right

        # -------- distance parallel and perpendicular to 
        dpara =  ux.*sx + uy.*sy .- length
        dperp = -uy.*sx + ux.*sy

        dpara[abs.(dpara).-length .<0] .= 0
        dpara = abs.(dpara)
        dpara .= min.(abs.(dpara.-length),dpara)

        rodimage = exp.(-(dperp.^2+dpara.^2)./(2*sig.^2))

        return rodimage
    end
    
    function corner_WST(inputs)
        sig, del = inputs
        image = max.(rod_half(0,0,20,sig+del/2,6),rod_half(0,0,20,sig-del/2,6))
        WST = DHC_compute(image, filter_hash)
        power = sum(image.^2)
        return WST, [sig, del, power]
    end
end
        
rod_DHC_out = @showprogress pmap(corner_WST, Iterators.product(0:1:360,0:1:360))
coeff = getindex.(rod_DHC_out, 1);
labels = getindex.(rod_DHC_out, 2);

coeff_out = hcat(coeff...)
labels_out = hcat(labels...)

h5write("../scratch_AKS/paper_data/corner_rod_S2.h5", "main/coeffs", coeff_out)
h5write("../scratch_AKS/paper_data/corner_rod_S2.h5", "main/labels", labels_out)

Progress: 100%|█████████████████████████████████████████| Time: 2:19:52m59


In [6]:
@everywhere begin
    using Statistics
    using LinearAlgebra
    using Distributed
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using Images
    using Interpolations
    using DSP
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function circle_interp2(xcen, ycen, length, pa, fwhm, bent; pivot = 0.5)
        nx = 256
        xbox = 1:nx
        ybox = 1:nx
        xbox = xbox.-nx/2 .+xcen
        ybox = ybox.-nx/2 .+ycen

        sx  = xbox' .* ones(size(ybox)[1])
        sy  = ones(size(xbox)[1])' .* ybox

        sig = fwhm/2.355
        dtor = pi/180
        ux = -sin(pa*dtor)   # 90 deg is up
        uy = cos(pa*dtor)   # 0 deg to the right

        start = [xcen-length,ycen]
        stop = [xcen+length,ycen]

        rotMat = [ux -uy; uy ux]

        pos_val = zeros(101,2)
        for (x_ind, x) in enumerate(0:0.01:1)
            pos_val[x_ind,:] .= rotMat*(getPosOnBentLine(start,stop,x,bent,pivot).-128).+128
        end

        mind = ones(nx,nx).*nx^3
        temp = zeros(nx,nx)

        for indx = 1:size(pos_val)[1]
            temp .= (sx.-pos_val[indx,1]).^2 .+ (sy.-pos_val[indx,2]).^2
            mind .= min.(mind,temp)
        end

        rodimage = exp.(-(mind)./(2*sig.^2))

        return rodimage
    end
    
    function perp(val)
        return (-val[2], val[1])
    end

    function getPosOnBentLine(lineStart, lineEnd, t, bendFactor, pivot)
        lineDir = lineEnd - lineStart
        lineLength = sqrt(sum(lineDir.^2))
        circleRad = lineLength / (bendFactor * 2 * pi);
        circleCenter = lineStart .+  (lineEnd .- lineStart)  .* pivot

        angle = pi + bendFactor * (t-pivot) * 2 * pi;
        posOnCircle = circleCenter .+ [cos(angle), sin(angle)] .* circleRad .+ [circleRad,0];

        return posOnCircle
    end
    
    function corner_WST(inputs)
        sig, del = inputs
        image = circle_interp2(128,128,20,sig,6,del+1e-5)
        WST = DHC_compute(image, filter_hash)
        power = sum(image.^2)
        return WST, [sig, del, power]
    end
end
        
rod_DHC_out = @showprogress pmap(corner_WST, Iterators.product(0:1:360,range(0,stop=1,length=361)))
coeff = getindex.(rod_DHC_out, 1);
labels = getindex.(rod_DHC_out, 2);

coeff_out = hcat(coeff...)
labels_out = hcat(labels...)

h5write("../scratch_AKS/paper_data/curve_rod_S2.h5", "main/coeffs", coeff_out)
h5write("../scratch_AKS/paper_data/curve_rod_S2.h5", "main/labels", labels_out)

Progress:   0%|▏                                        |  ETA: 4:31:09m55m

LoadError: InterruptException:

Progress:   0%|▏                                        |  ETA: 4:42:58

In [7]:
@everywhere begin
    using Statistics
    using LinearAlgebra
    using Distributed
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main/")
    using DHC_2DUtils
    using FFTW
    using Images
    using Interpolations
    using DSP
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function circle_a(xcen, ycen, length, pa, fwhm)
        nx = 256
        xbox = 1:nx
        ybox = 1:nx
        xbox = xbox.-nx/2 .+xcen
        ybox = ybox.-nx/2 .+ycen

        sx  = xbox' .* ones(size(ybox)[1])
        sy  = ones(size(xbox)[1])' .* ybox

        sig = fwhm/2.355

        r = sqrt.(sx.^2 + sy.^2)

        r[r.-length.<0] .= 0
        r .= min.(abs.(r.-length),r)

        rodimage = exp.(-(r.^2)./(2*sig.^2))

        return rodimage
    end
    
    function corner_WST(inputs)
        i, j = inputs
        image = circle_a(0,0,i,0,j)
        WST = DHC_compute(image, filter_hash)
        power = sum(image.^2)
        return WST, [i, j, power]
    end
end
        
rod_DHC_out = @showprogress pmap(corner_WST, Iterators.product(2 .^(range(-1,stop=6,length=361)),2 .^(range(-1,stop=6,length=361))))
coeff = getindex.(rod_DHC_out, 1);
labels = getindex.(rod_DHC_out, 2);

coeff_out = hcat(coeff...)
labels_out = hcat(labels...)

h5write("../scratch_AKS/paper_data/disc_data_log_S2.h5", "main/coeffs", coeff_out)
h5write("../scratch_AKS/paper_data/disc_data_log_S2.h5", "main/labels", labels_out)

Progress:   0%|▏                                        |  ETA: 2:31:38m20m

LoadError: InterruptException:

Progress:   0%|▏                                        |  ETA: 2:30:39